<h1 style="font-size:3rem;color:blue;">Python Analysis Practice of Single-Cell RNAseq Data</h1>

# Data Source

In [ ]:
# All data used in this exercise has been sourced from: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE171524
# This script is primarily used for data analysis practice for GitHub user CPalmer3200

# Installing scanpy module and importing data

In [ ]:
# !pip install scanpy

In [1]:
import scanpy as sc

In [ ]:
data1 = sc.read_csv('C:/Users/chris/Single cell practice/GSM5226574_C51ctr_raw_counts.csv').T
data1

# Pre-processing - doublet removal

In [ ]:
# Initial stage to pre-processing is doublet removal
# This will allow the computer to predict doublets (when two or more cells are mistaken as a single cell)
# This should be done on individual samples instead of integrated samples

In [ ]:
# !pip install scvi-tools

In [ ]:
# !pip install torchaudio

In [2]:
import scvi
scvi.settings.seed = 0

C:\ProgramData\anaconda3\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\ProgramData\anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
C:\Users\chris\AppData\Roaming\Python\Python311\site-packages\scvi\_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
C:\Users\chris\AppData\Roaming\Python\Python311\site-packages\scvi\_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please 

In [ ]:
# Filter genes out that appear in fewer than 10 cells
sc.pp.filter_genes(data1, min_cells = 10)

In [ ]:
data1

In [ ]:
# !pip install --user scikit-misc

In [ ]:
# Keep 2000 top variable genes (2000 genes that describe the data the best)
sc.pp.highly_variable_genes(data1, n_top_genes = 2000, subset = True, flavor = 'seurat_v3')

In [ ]:
# Setup SCVI model with default parameters and train
scvi.model.SCVI.setup_anndata(data1)
vae = scvi.model.SCVI(data1)
vae.train()

In [ ]:
# Create solo model which predicts doublets and pass the vae model through
solo = scvi.external.SOLO.from_scvi_model(vae)
solo.train()

In [ ]:
# Return df and make new column with predicted label
df = solo.predict()
df['prediction'] = solo.predict(soft = False)

df

In [ ]:
#Count the predictions
df.groupby('prediction').count()

In [ ]:
#Assign the predicted doublets to their own df - this will be used to filter data1
doublets = df[(df.prediction == 'doublet')]
doublets

In [ ]:
# Reload data1 variable
data1 = sc.read_csv('C:/Users/chris/Single cell practice/GSM5226574_C51ctr_raw_counts.csv').T
data1

In [ ]:
# Add binary variable to check if data1 objects are predicted doublets
data1.obs['doublet'] = data1.obs.index.isin(doublets.index)
data1.obs

In [ ]:
# Filter out the predicted doublets that appear true (DOUBLET REMOVAL COMPLETE)
data1 = data1[~data1.obs.doublet]

# Pre-processing - labelling of mitochondrial genes

In [ ]:
# Mitochondrial genes start with MT- so use python to filter these
data1.var['mt'] = data1.var.index.str.startswith('MT-')
data1.var

# Pre-processing - labelling ribosomal genes

In [9]:
import pandas as pd

In [10]:
#Import list of ribosomal gene names for filtering
ribo_url = "http://software.broadinstitute.org/gsea/msigdb/download_geneset.jsp?geneSetName=KEGG_RIBOSOME&fileType=txt"

In [11]:
# Format list of ribosomal genes
ribo_genes = pd.read_table(ribo_url, skiprows=2, header=None)
ribo_genes

,0
0,FAU
1,MRPL13
2,RPL10
3,RPL10A
4,RPL10L
...,...
83,RPS9
84,RPSA
85,RSL24D1
86,RSL24D1P11


In [ ]:
# Check if genes in data1 are present in the ribo_genes list
data1.var['ribo'] = data1.var_names.isin(ribo_genes[0].values)
data1.var

In [ ]:
# Generate quality control statistics from scanpy
sc.pp.calculate_qc_metrics(data1, qc_vars=['mt', 'ribo'], percent_top = None, log1p = False, inplace = True)
data1.var

In [ ]:
#Mitochondrial/Ribosomal count and percentages are now available in the observations df
data1.obs

In [ ]:
# Filter out genes that were in fewer than 3 cells
sc.pp.filter_genes(data1, min_cells = 3)

# Sort the data by n_cells_by_counts
data1.var.sort_values('n_cells_by_counts')

In [ ]:
# This dataset has likely already been filtered for total_counts of genes - this should be checked before proceeding
data1.obs.sort_values('total_counts')

In [ ]:
# Plot the quality control metrics to look for outliers to exclude
sc.pl.violin(data1, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo'], jitter=0.4, multi_panel=True)

In [4]:
# Use numpy to find and filter the 98 percentile value
import numpy as np

In [ ]:
upper_lim = np.quantile(data1.obs.n_genes_by_counts.values, .98)
upper_lim

In [ ]:
data1 = data1[data1.obs.n_genes_by_counts < upper_lim]
data1.obs

In [ ]:
# Filter out the top 20% of pct_counts_mt
data1 = data1[data1.obs.pct_counts_mt < 20]
data1.obs

In [ ]:
#Filter out pct_counts_ribo above the value 2 (check the violin plot)
data1 = data1[data1.obs.pct_counts_ribo < 2]
data1.obs

In [ ]:
# The data has now been cleaned for doublets and outliers and is ready for clustering and analysis
data1

# Normalisation

In [ ]:
# Permits comparison of cells and genes by normalising the data to account for biases arising from sequencing, etc
data1.X.sum(axis = 1)

In [ ]:
# Normalise every cell to 10,000 UMI
sc.pp.normalize_total(data1, target_sum=1e4)

In [ ]:
# The data has now been normalised
data1.X.sum(axis = 1)

# Change to log counts and recall
sc.pp.log1p(data1)
data1.X.sum(axis = 1)

In [ ]:
# Data is now comparable and must be saved in this instance - many functions use this .raw data
data1.raw = data1

# Clustering

In [ ]:
# Find 2000 top highly variable genes - adds in stats about gene variability
sc.pp.highly_variable_genes(data1, n_top_genes=2000)
data1.var

In [ ]:
# Highly variable genes can be plotted easily using scanpy function
sc.pl.highly_variable_genes(data1)

In [ ]:
# Filter out not highly variable genes - this does not impact the .raw data save
data1 = data1[:, data1.var.highly_variable]
data1

In [ ]:
# Regress out the differences arising from total counts, mitochondrial counts and ribosomal counts
# This removes variations in the data that are due to processing and sample quality
sc.pp.regress_out(data1, ['total_counts', 'pct_counts_mt', 'pct_counts_ribo'])
data1

In [ ]:
# Normalise each gene to the unit variance of each gene
sc.pp.scale(data1, max_value=10)

In [ ]:
# Run PCA (Principle Component Analysis) to reduce the data dimensions
sc.tl.pca(data1, svd_solver='arpack')
# Neighbourhood looks at distance and creates a matrix which is used for clustering
sc.pp.neighbors(data1, n_pcs=30)

In [ ]:
# Use umap to project the data from ~30 dimensions to just 2 - each point is a single cell
sc.tl.umap(data1)

In [ ]:
sc.pl.umap(data1)

In [ ]:
# Assigning clusters requires the leiden algorithm
# !pip install leidenalg

In [ ]:
#Run the leiden algorithm to generate clusters - Resolution [0-1] where 0 is the least clusters and 1 is the most
sc.tl.leiden(data1, resolution=0.5)

In [ ]:
# Leiden column has now been added to the observations table
data1.obs

In [ ]:
# The cells can now be coloured based on the leiden label
sc.pl.umap(data1, color=['leiden'])

# Integration of multiple samples

In [5]:
# Function for doublet removal, pre-processing and formatting

def pp(csv_path):
    adata = sc.read_csv(csv_path).T
    sc.pp.filter_genes(adata, min_cells = 10)
    sc.pp.highly_variable_genes(adata, n_top_genes = 2000, subset = True, flavor = 'seurat_v3')
    scvi.model.SCVI.setup_anndata(adata)
    vae = scvi.model.SCVI(adata)
    vae.train()
    solo = scvi.external.SOLO.from_scvi_model(vae)
    solo.train()
    df = solo.predict()
    df['prediction'] = solo.predict(soft = False)
    df.index = df.index.map(lambda x: x[:-2])
    df['dif'] = df.doublet - df.singlet
    doublets = df[(df.prediction == 'doublet') & (df.dif > 1)]
    
    adata = sc.read_csv(csv_path).T
    adata.obs['Sample'] = csv_path.split('_')[1] #'/GSM5226574_C51ctr_raw_counts.csv'
    
    adata.obs['doublet'] = adata.obs.index.isin(doublets.index)
    adata = adata[~adata.obs.doublet]
    
    
    sc.pp.filter_cells(adata, min_genes=200) #get rid of cells with fewer than 200 genes
    #sc.pp.filter_genes(adata, min_cells=3) #get rid of genes that are found in fewer than 3 cells
    adata.var['mt'] = adata.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
    adata.var['ribo'] = adata.var_names.isin(ribo_genes[0].values)
    sc.pp.calculate_qc_metrics(adata, qc_vars=['mt', 'ribo'], percent_top=None, log1p=False, inplace=True)
    upper_lim = np.quantile(adata.obs.n_genes_by_counts.values, .98)
    adata = adata[adata.obs.n_genes_by_counts < upper_lim]
    adata = adata[adata.obs.pct_counts_mt < 20]
    adata = adata[adata.obs.pct_counts_ribo < 2]

    return adata

In [6]:
import os

In [7]:
# Function for only taking .csv files in the directory, processing with pp function, and then appending to output

def integrate_csv_data(directory):
    out = []
    for file in os.listdir(directory):
        if file.endswith('.csv'):
            csv_path = os.path.join(directory, file)
            out.append(pp(csv_path))
    return out

In [12]:
# Run the data integration and save to the variable data2
data2 = integrate_csv_data("C:/Users/chris/Single cell practice/")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Epoch 400/400: 100%|█████████████| 400/400 [10:33<00:00,  1.68s/it, v_num=1, train_loss_step=355, train_loss_epoch=323]

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|█████████████| 400/400 [10:33<00:00,  1.58s/it, v_num=1, train_loss_step=355, train_loss_epoch=323]
INFO     Creating doublets, preparing SOLO model.                                                                  


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


...

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|█████████████| 400/400 [09:44<00:00,  1.46s/it, v_num=1, train_loss_step=305, train_loss_epoch=306]
INFO     Creating doublets, preparing SOLO model.                                                                  


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Epoch 159/400:  40%|███▌     | 159/400 [00:44<01:06,  3.61it/s, v_num=1, train_loss_step=0.177, train_loss_epoch=0.227]
Monitored metric validation_loss did not improve in the last 30 records. Best score: 0.238. Signaling Trainer to stop.


C:\Users\chris\AppData\Roaming\Python\Python311\site-packages\scanpy\preprocessing\_simple.py:139: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_genes'] = number


In [13]:
data2

[View of AnnData object with n_obs × n_vars = 5960 × 34546
     obs: 'Sample', 'doublet', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo'
     var: 'mt', 'ribo', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts',
 View of AnnData object with n_obs × n_vars = 4415 × 34546
     obs: 'Sample', 'doublet', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo'
     var: 'mt', 'ribo', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts',
 View of AnnData object with n_obs × n_vars = 6928 × 34546
     obs: 'Sample', 'doublet', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo'
     var: 'mt', 'ribo', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts',
 View of AnnData object with n_obs × n_vars = 4284 × 34546

In [14]:
# Concatonate the datasets into a single object 
data = sc.concat(data2)
data

AnnData object with n_obs × n_vars = 27152 × 34546
    obs: 'Sample', 'doublet', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo'

In [15]:
# Filter out genes present in <10 cells
sc.pp.filter_genes(data, min_cells=10)
data.X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [16]:
#Convert to sparse matrix to reduce strain on memory
from scipy.sparse import csr_matrix

In [17]:
data.X = csr_matrix(data.X)
data.X

<27152x24995 sparse matrix of type '<class 'numpy.float32'>'
	with 22706309 stored elements in Compressed Sparse Row format>

In [18]:
# Save the data
data.write_h5ad('combined.h5ad')

In [19]:
# Read the saved data file
data = sc.read_h5ad('combined.h5ad')
data

AnnData object with n_obs × n_vars = 27152 × 24995
    obs: 'Sample', 'doublet', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo'
    var: 'n_cells'

In [20]:
data.obs.groupby('Sample').count()

,doublet,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_ribo,pct_counts_ribo
Sample,,,,,,,,
C51ctr,5960,5960,5960,5960,5960,5960,5960,5960
C52ctr,4415,4415,4415,4415,4415,4415,4415,4415
C53ctr,6928,6928,6928,6928,6928,6928,6928,6928
C54ctr,4284,4284,4284,4284,4284,4284,4284,4284
C55ctr,5565,5565,5565,5565,5565,5565,5565,5565
